<a href="https://colab.research.google.com/github/LikeWind99/DeepLearning/blob/master/%E4%BA%8C%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch as t
import torchvision
from torchvision.datasets import ImageFolder
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tqdm
import numpy as np

In [7]:
normalize = T.Normalize(mean=[0.4, 0.4, 0.4], std=[0.2, 0.2, 0.2])
transform = T.Compose([
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    normalize
])
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
batch_size = 4
num_works = 2
epochs = 100


In [10]:
dataset = ImageFolder(r'/content/drive/MyDrive/test', transform=transform)
sample_size = len(dataset)
trainDataset, testDataset = t.utils.data.random_split(dataset= dataset, lengths=[int(0.8 * sample_size), int(0.2 * sample_size)])

trainLoader = DataLoader(dataset=trainDataset, batch_size=batch_size, shuffle=True, drop_last=True)
testLoader = DataLoader(dataset=testDataset, batch_size=batch_size, drop_last=True)

In [11]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResidualBlock,self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel,outchannel,3,stride, 1,bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel,outchannel,3,1,1,bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.right = shortcut
    
    def forward(self,x):
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)
    
class ResNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet,self).__init__()
        self.pre = nn.Sequential(
                nn.Conv2d(3, 64, 7, 2, 3, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(3, 2, 1)
        )
        self.layer1 = self._make_layer( 64, 64, 3)
        self.layer2 = self._make_layer( 64, 128, 4, stride=2)
        self.layer3 = self._make_layer( 128, 256, 6, stride=2)
        self.layer4 = self._make_layer( 256, 512, 3, stride=2)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self,  inchannel, outchannel, block_num, stride=1):
        shortcut = nn.Sequential(
                nn.Conv2d(inchannel,outchannel,1,stride, bias=False),
                nn.BatchNorm2d(outchannel)
        )
        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))
        
        for i in range(1, block_num):
            layers.append(ResidualBlock(outchannel, outchannel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.pre(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = F.avg_pool2d(x, 7)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [14]:
net = ResNet()
total_step = len(trainLoader)
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.Adam(params=net.parameters())
for epoch in range(epochs):
  for i, (imgs, labels) in tqdm.tqdm(enumerate(trainLoader)):
    net = net.to(device)
    optimizer.zero_grad()
    imgs = imgs.to(device)
    labels = labels.to(device)
    predict = net(imgs)
    loss = criterion(predict, labels)
    loss.backward()
    optimizer.step()
    if (i + 1) % 100 == 0:
      print(f'Epoch [{epoch}/{epochs}], Step [{i+1}/{total_step}], loss: {loss.item():.4f}')





0it [00:00, ?it/s]


1it [00:02,  2.26s/it]


2it [00:04,  2.29s/it]


3it [00:07,  2.35s/it]


4it [00:08,  2.15s/it]


5it [00:10,  2.00s/it]


6it [00:12,  2.06s/it]


7it [00:15,  2.42s/it]


8it [00:17,  2.17s/it]


9it [00:20,  2.43s/it]


10it [00:22,  2.18s/it]


11it [00:25,  2.44s/it]


12it [00:28,  2.65s/it]


13it [00:31,  2.74s/it]


14it [00:33,  2.59s/it]


15it [00:36,  2.72s/it]


16it [00:39,  2.80s/it]


17it [00:42,  2.94s/it]


18it [00:45,  2.97s/it]


19it [00:48,  2.99s/it]


20it [00:51,  2.76s/it]


21it [00:54,  2.85s/it]


22it [00:56,  2.68s/it]


23it [00:58,  2.59s/it]


24it [01:01,  2.75s/it]


25it [01:04,  2.80s/it]


26it [01:06,  2.43s/it]


27it [01:08,  2.36s/it]


28it [01:11,  2.55s/it]


29it [01:14,  2.72s/it]


30it [01:17,  2.59s/it]


31it [01:19,  2.52s/it]


32it [01:22,  2.65s/it]


33it [01:24,  2.41s/it]


34it [01:26,  2.39s/it]


35it [01:28,  2.13s/it]


36it [01:31,  2.41s/it]


37it [01:32,  2.23s/it]


38it [01:35,  2.25s/it]

Epoch [0/100], Step [100/187], loss: 0.0229





101it [04:11,  2.10s/it]


102it [04:13,  1.95s/it]


103it [04:16,  2.27s/it]


104it [04:18,  2.27s/it]


105it [04:20,  2.04s/it]


106it [04:22,  2.09s/it]


107it [04:24,  2.15s/it]


109it [04:27,  1.96s/it]


110it [04:30,  2.31s/it]


111it [04:32,  2.13s/it]


112it [04:34,  2.14s/it]


113it [04:37,  2.23s/it]


114it [04:40,  2.46s/it]


115it [04:41,  2.16s/it]


116it [04:44,  2.22s/it]


117it [04:46,  2.40s/it]


118it [04:48,  2.29s/it]


119it [04:51,  2.32s/it]


120it [04:54,  2.58s/it]


121it [04:57,  2.74s/it]


122it [04:58,  2.18s/it]


123it [05:00,  2.20s/it]


124it [05:02,  2.15s/it]


125it [05:05,  2.41s/it]


126it [05:08,  2.37s/it]


127it [05:11,  2.64s/it]


128it [05:12,  2.34s/it]


129it [05:16,  2.59s/it]


130it [05:17,  2.29s/it]


131it [05:20,  2.52s/it]


132it [05:24,  2.73s/it]


133it [05:26,  2.62s/it]


134it [05:29,  2.76s/it]


135it [05:31,  2.60s/it]


136it [05:34,  2.53s/it]


137it [05:36,  2.47s/it]


138it [05:38,  2.43s/it]


Epoch [1/100], Step [100/187], loss: 0.0851





107it [00:04, 22.47it/s]


110it [00:04, 22.78it/s]


113it [00:04, 22.96it/s]


116it [00:05, 22.31it/s]


119it [00:05, 22.63it/s]


122it [00:05, 22.69it/s]


125it [00:05, 22.84it/s]


128it [00:05, 23.24it/s]


131it [00:05, 23.36it/s]


134it [00:05, 23.34it/s]


137it [00:06, 23.38it/s]


140it [00:06, 22.79it/s]


143it [00:06, 23.07it/s]


146it [00:06, 23.37it/s]


149it [00:06, 23.02it/s]


152it [00:06, 23.27it/s]


155it [00:06, 23.37it/s]


158it [00:06, 22.89it/s]


161it [00:07, 23.08it/s]


164it [00:07, 23.26it/s]


167it [00:07, 23.14it/s]


170it [00:07, 23.23it/s]


173it [00:07, 23.04it/s]


176it [00:07, 23.18it/s]


179it [00:07, 22.83it/s]


182it [00:07, 23.13it/s]


187it [00:08, 22.86it/s]



0it [00:00, ?it/s]


3it [00:00, 24.57it/s]


6it [00:00, 23.89it/s]


9it [00:00, 23.37it/s]


12it [00:00, 23.41it/s]


15it [00:00, 23.31it/s]


18it [00:00, 23.37it/s]


21it [00:00, 22.67it/s]


24it [00:01, 22.87it/s]


27it [00:01, 23.18it/s]


30it [00:01, 23

...




105it [00:04, 22.45it/s]


108it [00:04, 21.97it/s]


111it [00:04, 22.32it/s]


114it [00:04, 22.83it/s]


117it [00:05, 23.12it/s]


120it [00:05, 23.27it/s]


123it [00:05, 23.42it/s]


126it [00:05, 23.57it/s]


129it [00:05, 23.47it/s]


132it [00:05, 23.03it/s]


135it [00:05, 23.27it/s]


138it [00:05, 23.18it/s]


141it [00:06, 22.83it/s]


144it [00:06, 23.12it/s]


147it [00:06, 23.35it/s]


150it [00:06, 23.21it/s]


153it [00:06, 23.36it/s]


156it [00:06, 23.12it/s]


159it [00:06, 23.05it/s]


162it [00:06, 23.32it/s]


165it [00:07, 23.44it/s]


168it [00:07, 23.31it/s]


171it [00:07, 23.32it/s]


174it [00:07, 23.25it/s]


177it [00:07, 23.13it/s]


180it [00:07, 23.07it/s]


183it [00:07, 23.22it/s]


187it [00:08, 23.10it/s]



0it [00:00, ?it/s]


3it [00:00, 24.05it/s]


6it [00:00, 23.81it/s]


9it [00:00, 23.71it/s]


12it [00:00, 23.60it/s]


15it [00:00, 23.29it/s]


18it [00:00, 23.41it/s]


21it [00:00, 23.40it/s]


24it [00:01, 23.34it/s]


27it [00:01, 2

Epoch [98/100], Step [100/187], loss: 0.0413





105it [00:04, 22.92it/s]


108it [00:04, 22.37it/s]


111it [00:04, 22.60it/s]


114it [00:04, 22.63it/s]


117it [00:05, 22.94it/s]


120it [00:05, 23.26it/s]


123it [00:05, 23.56it/s]


126it [00:05, 23.59it/s]


129it [00:05, 23.68it/s]


132it [00:05, 23.58it/s]


135it [00:05, 23.50it/s]


138it [00:05, 23.59it/s]


141it [00:06, 23.67it/s]


144it [00:06, 23.56it/s]


147it [00:06, 23.65it/s]


150it [00:06, 23.48it/s]


153it [00:06, 23.53it/s]


156it [00:06, 23.31it/s]


159it [00:06, 23.12it/s]


162it [00:06, 23.34it/s]


165it [00:07, 23.54it/s]


168it [00:07, 23.42it/s]


171it [00:07, 23.54it/s]


174it [00:07, 23.68it/s]


177it [00:07, 22.98it/s]


180it [00:07, 22.92it/s]


183it [00:07, 23.05it/s]


187it [00:08, 23.15it/s]



0it [00:00, ?it/s]


3it [00:00, 24.42it/s]


6it [00:00, 24.29it/s]


9it [00:00, 24.03it/s]


12it [00:00, 23.98it/s]


15it [00:00, 23.71it/s]


18it [00:00, 23.74it/s]


21it [00:00, 23.69it/s]


24it [00:01, 23.78it/s]


27it [00:01, 2

Epoch [99/100], Step [100/187], loss: 0.0066





105it [00:04, 22.85it/s]


108it [00:04, 23.03it/s]


111it [00:04, 22.75it/s]


114it [00:04, 22.85it/s]


117it [00:05, 23.02it/s]


120it [00:05, 23.05it/s]


123it [00:05, 23.21it/s]


126it [00:05, 23.31it/s]


129it [00:05, 23.34it/s]


132it [00:05, 22.71it/s]


135it [00:05, 22.72it/s]


138it [00:05, 22.20it/s]


141it [00:06, 22.62it/s]


144it [00:06, 23.06it/s]


147it [00:06, 23.03it/s]


150it [00:06, 23.30it/s]


153it [00:06, 23.38it/s]


156it [00:06, 23.44it/s]


159it [00:06, 23.57it/s]


162it [00:06, 23.42it/s]


165it [00:07, 23.45it/s]


168it [00:07, 23.50it/s]


171it [00:07, 23.48it/s]


174it [00:07, 23.58it/s]


177it [00:07, 23.72it/s]


180it [00:07, 23.55it/s]


183it [00:07, 23.61it/s]


187it [00:08, 23.31it/s]


In [15]:
with t.no_grad():
    correct = 0
    total = 0
    for img, label in testLoader:
        img = img.to(device)
        label = label.to(device)
        outputs = net(img)
        _, predicted = t.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 92.85714285714286 %
Accuracy of the model on the test images: 93.75 %
Accuracy of the model on the test images: 94.44444444444444 %
Accuracy of the model on the test images: 95.0 %
Accuracy of the model on the test images: 95.45454545454545 %
Accuracy of the model on the test images: 95.83333333333333 %
Accuracy of the model on the test images: 96.15384615384616 %
Accuracy of the model on the test images: 92.85714285714286 %
Accuracy of the model on the test images: 93.33333333333333 %
Accuracy of the model on the test images: 93.75 %
Accuracy of the model on the test images: 94.11764705882354 %
Accuracy of the model on the test images: 94.4444444444